This code is written in MATLAB and it calculates the mode shapes and natural frequencies of a 3D frame with 12 elements, each has two nodes with 6 DOFs (3 displacements and 3 rotations).
Four nodes on one side of the frame are attached to 3 springs to limit the frame's free motion in x,y, and z directions as the boundary condition.

In [2]:
clc
clear all
n=12; % number of elements (can't be changed!)

le=0.4; % length of each connector in the fram (m)
ro=7900; % Density (kg/m^3)
A=1.0000e-004; % Cross-sectional are of connectors (m^2)
j=1.6667e-009; % m^4
E=2e11; % Young's modulus
G=79e9 ; % Shear modulus
Ixx=833.3333333333e-12; %m^4
Iyy=833.3333333333e-12; %m^4
Izz=833.3333333333e-12; %m^4
le(1:n)=le; % Elements are assumed to have the same length

% ---------------------- Mass and Stiffness matrix for each element of the frame
for i=1:n
    Me_local(:,:,i)=ro*A*le(i)*[1/3 0 0 0 0 0 1/6 0 0 0 0 0; ...
        0 13/35 0 0 0 11*le(i)/210 0 9/70 0 0 0 -13*le(i)/420; ...
        0 0 13/35 0 -11*le(i)/210 0 0 0 9/70 0 13*le(i)/420 0; ...
        0 0 0 j/(3*A) 0 0 0 0 0 j/(6*A) 0 0; ...
        0 0 -11*le(i)/210 0 le(i)^2/105 0 0 0 -13*le(i)/420 0 -le(i)^2/140 0; ...
        0 11*le(i)/210 0 0 0 le(i)^2/105 0 13*le(i)/420 0 0 0 -le(i)^2/140; ...
        1/6 0 0 0 0 0 1/3 0 0 0 0 0; ...
        0 9/70 0 0 0 13*le(i)/420 0 13/35 0 0 0 -11*le(i)/210; ...
        0 0 9/70 0 -13*le(i)/420 0 0 0 13/35 0 11*le(i)/210 0; ...
        0 0 0 j/(6*A) 0 0 0 0 0 j/(3*A) 0 0; ...
        0 0 13*le(i)/420 0 -le(i)^2/140 0 0 0 11*le(i)/210 0 le(i)^2/105 0; ...
        0 -13*le(i)/420 0 0 0 -le(i)^2/140 0 -11*le(i)/210 0 0 0 le(i)^2/105];
    ke_local(:,:,i)=[(E*A)/le(i) 0 0 0 0 0 -(E*A)/le(i) 0 0 0 0 0; ...
        0 (12*E*Izz)/le(i)^3 0 0 0 (6*E*Izz)/le(i)^2 0 (-12*E*Izz)/le(i)^3 0 0 0 (6*E*Izz)/le(i)^2; ...
        0 0 (12*E*Iyy)/le(i)^3 0 (-6*E*Iyy)/le(i)^2 0 0 0 (-12*E*Iyy)/le(i)^3 0 (-6*E*Iyy)/le(i)^2 0; ...
        0 0 0 G*j/le(i) 0 0 0 0 0 -G*j/le(i) 0 0; ...
        0 0 (-6*E*Iyy)/le(i)^2 0 (4*E*Iyy)/le(i) 0 0 0 (6*E*Iyy)/le(i)^2 0 (2*E*Iyy)/le(i) 0; ...
        0 (6*E*Izz)/le(i)^2 0 0 0 (4*E*Izz)/le(i) 0 (-6*E*Izz)/le(i)^2 0 0 0 (2*E*Izz)/le(i); ...
        -E*A/le(i) 0 0 0 0 0 E*A/le(i) 0 0 0 0 0; ...
        0 (-12*E*Izz)/le(i)^3 0 0 0 (-6*E*Izz)/le(i)^2 0 (12*E*Izz)/le(i)^3 0 0 0 (-6*E*Izz)/le(i)^2; ...
        0 0 (-12*E*Iyy)/le(i)^3 0 (6*E*Iyy)/le(i)^2 0 0 0 (12*E*Iyy)/le(i)^3 0 (6*E*Iyy)/le(i)^2 0; ...
        0 0 0 -G*j/le(i) 0 0 0 0 0 G*j/le(i) 0 0; ...
        0 0 (-6*E*Iyy)/le(i)^2 0 (2*E*Iyy)/le(i) 0 0 0 (6*E*Iyy)/le(i)^2 0 (4*E*Iyy)/le(i) 0; ...
        0 (6*E*Izz)/le(i)^2 0 0 0 (2*E*Izz)/le(i) 0 (-6*E*Izz)/le(i)^2 0 0 0 (4*E*Izz)/le(i)];
end
lambd(:,:,1)=[0 0 1;0 1 0;-1 0 0];
lambd(:,:,2)=[1 0 0;0 1 0;0 0 1];
lambd(:,:,12)=[0 1 0;-1 0 0;0 0 1];
lambd(:,:,3)=[0 0 -1;0 -1 0;-1 0 0];
lambd(:,:,4)=lambd(:,:,2);
lambd(:,:,5)=lambd(:,:,1);
lambd(:,:,6)=lambd(:,:,2);
lambd(:,:,7)=lambd(:,:,3);
lambd(:,:,8)=lambd(:,:,2);
lambd(:,:,9)=lambd(:,:,12);
lambd(:,:,10)=lambd(:,:,12);
lambd(:,:,11)=lambd(:,:,12);
for i=1:n
    lambda(:,:,i)=[lambd(:,:,i) zeros(3,9);zeros(3,3) lambd(:,:,i) zeros(3,6);zeros(3,6) lambd(:,:,i) ...
        zeros(3,3);zeros(3,9) lambd(:,:,i)];
end
for i=1:n
    Ke(:,:,i)=lambda(:,:,i)'*ke_local(:,:,i)*lambda(:,:,i);
    Me(:,:,i)=lambda(:,:,i)'*Me_local(:,:,i)*lambda(:,:,i);
end

%-------------------------------- assembly  proceess -------------------------------
nodes=8;
dof=nodes*6;
K=[Ke(:,:,1) zeros(12,dof-12);zeros(dof-12,dof)];
M=[Me(:,:,1) zeros(12,dof-12);zeros(dof-12,dof)];

for i=1:2
    K=K+[zeros(6*i,dof);zeros(12,6*i) Ke(:,:,i+1) zeros(12,dof-12-(6*i));zeros(dof-12-(6*i),dof)];
    M=M+[zeros(6*i,dof);zeros(12,6*i) Me(:,:,i+1) zeros(12,dof-12-(6*i));zeros(dof-12-(6*i),dof)];
end

for i=4:6
    K=K+[zeros(6*i,dof);zeros(12,6*i) Ke(:,:,i+1) zeros(12,dof-12-(6*i));zeros(dof-12-(6*i),dof)];
    M=M+[zeros(6*i,dof);zeros(12,6*i) Me(:,:,i+1) zeros(12,dof-12-(6*i));zeros(dof-12-(6*i),dof)];
end

%elemenet 4
for i=1:6
    for j=1:6
        K(i,j)=K(i,j)+Ke(i,j,4);
        K(i,j+18)=K(i,j+18)+Ke(i,j+6,4);
        M(i,j)=M(i,j)+Me(i,j,4);
        M(i,j+18)=M(i,j+18)+Me(i,j+6,4);
        
        K(i+18,j+18)=K(i+18,j+18)+Ke(i+6,j+6,4);
        K(i+18,j)=K(i+18,j)+Ke(i+6,j,4);
        M(i+18,j+18)=M(i+18,j+18)+Me(i+6,j+6,4);
        M(i+18,j)=M(i+18,j)+Me(i+6,j,4);
    end
end

%element 8
for i=1:6
    for j=1:6
        K(i+24,j+24)=K(i+24,j+24)+Ke(i,j,8);
        K(i+24,j+42)=K(i+24,j+42)+Ke(i,j+6,8);
        M(i+24,j+24)=M(i+24,j+24)+Me(i,j,8);
        M(i+24,j+42)=M(i+24,j+42)+Me(i,j+6,8);
        
        K(i+42,j+42)=K(i+42,j+42)+Ke(i+6,j+6,8);
        K(i+42,j+24)=K(i+42,j+24)+Ke(i+6,j,8);
        M(i+42,j+42)=M(i+42,j+42)+Me(i+6,j+6,8);
        M(i+42,j+24)=M(i+42,j+24)+Me(i+6,j,8);
    end
end

%elemenet 9
for i=1:6
    for j=1:6
        K(i,j)=K(i,j)+Ke(i,j,9);
        K(i,j+24)=K(i,j+24)+Ke(i,j+6,9);
        M(i,j)=M(i,j)+Me(i,j,9);
        M(i,j+24)=M(i,j+24)+Me(i,j+6,9);
        
        K(i+24,j+24)=K(i+24,j+24)+Ke(i+6,j+6,9);
        K(i+24,j)=K(i+24,j)+Ke(i+6,j,9);
        M(i+24,j+24)=M(i+24,j+24)+Me(i+6,j+6,9);
        M(i+24,j)=M(i+24,j)+Me(i+6,j,9);
    end
end

%elemenet 10
for i=1:6
    for j=1:6
        K(i+6,j+6)=K(i+6,j+6)+Ke(i,j,10);
        K(i+6,j+30)=K(i+6,j+30)+Ke(i,j+6,10);
        M(i+6,j+6)=M(i+6,j+6)+Me(i,j,10);
        M(i+6,j+30)=M(i+6,j+30)+Me(i,j+6,10);
        
        K(i+30,j+30)=K(i+30,j+30)+Ke(i+6,j+6,10);
        K(i+30,j+6)=K(i+30,j+6)+Ke(i+6,j,10);
        M(i+30,j+30)=M(i+30,j+30)+Me(i+6,j+6,10);
        M(i+30,j+6)=M(i+30,j+6)+Me(i+6,j,10);
    end
end

%element 11
for i=1:6
    for j=1:6
        K(i+12,j+12)=K(i+12,j+12)+Ke(i,j,11);
        K(i+12,j+36)=K(i+12,j+36)+Ke(i,j+6,11);
        M(i+12,j+12)=M(i+12,j+12)+Me(i,j,11);
        M(i+12,j+36)=M(i+12,j+36)+Me(i,j+6,11);
        
        K(i+36,j+36)=K(i+36,j+36)+Ke(i+6,j+6,11);
        K(i+36,j+12)=K(i+36,j+12)+Ke(i+6,j,11);
        M(i+36,j+36)=M(i+36,j+36)+Me(i+6,j+6,11);
        M(i+36,j+12)=M(i+36,j+12)+Me(i+6,j,11);
    end
end

%element 12
for i=1:6
    for j=1:6
        K(i+18,j+18)=K(i+18,j+18)+Ke(i,j,12);
        K(i+18,j+42)=K(i+18,j+42)+Ke(i,j+6,12);
        M(i+18,j+18)=M(i+18,j+18)+Me(i,j,12);
        M(i+18,j+42)=M(i+18,j+42)+Me(i,j+6,12);
        
        K(i+42,j+42)=K(i+42,j+42)+Ke(i+6,j+6,12);
        K(i+42,j+18)=K(i+42,j+18)+Ke(i+6,j,12);
        M(i+42,j+42)=M(i+42,j+42)+Me(i+6,j+6,12);
        M(i+42,j+18)=M(i+42,j+18)+Me(i+6,j,12);
    end
end



%---------------------------- Boundry Condition --------------------------

kx_bb=1000;
ky_bb=5000;
kz_bb=2000;


%node 1
K(1,1)=K(1,1)+kx_bb;
K(2,2)=K(2,2)+ky_bb;
K(3,3)=K(3,3)+kz_bb;

%node 4
K(19,19)=K(19,19)+kx_bb;
K(20,20)=K(20,20)+ky_bb;
K(21,21)=K(21,21)+kz_bb;


%node 5
K(25,25)=K(25,25)+kx_bb;
K(26,26)=K(26,26)+ky_bb;
K(27,27)=K(27,27)+kz_bb;

%node 8
K(43,43)=K(43,43)+kx_bb;
K(44,44)=K(44,44)+ky_bb;
K(45,45)=K(45,45)+kz_bb;

% ------------------------------- Natural Freqeuncies ---------------
[phi,fr2]=eig(K,M); % Mode shapes
fr=sqrt(abs(fr2)); % Natural frequencies
disp('First 10 natural Frequencies are (Hz):')
NF = diag(fr/(2*pi));
disp(NF(1:10))

First 10 natural Frequencies are (Hz):
    3.7680
    4.3926
    7.3091
    7.9911
    9.8775
   15.1960
   62.3422
   62.6970
   63.5001
   64.0182
